In [1]:
import csv

# initializing the titles and rows list 
fields = [] 
rows = [] 
# raw_questions = []
raw_questions_to_hw = dict()

filename = 'contributions.csv'
# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
  
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 

    curr_question = ""
    curr_text = ""
    curr_topic = ""
    curr_folder = ""
    curr_answer = ""
    curr_number = 0
    curr_follow_up = 0
    endorsed = 0
    
    # This will help us take the most recent version because it always comes first
    already_filled = 0
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number" and curr_number != int(col)):
                curr_folders = curr_folder.split()
                for folder in curr_folders:
                    if folder[-1] == ';':
                            folder = folder[:-1]
                    if(curr_question != "" and curr_answer != ""):
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({curr_folder : raw_questions})
                curr_number = int(col)
                curr_follow_up = 0
                curr_question = ""
                already_filled = 0
                curr_answer = ""
                curr_topic = ""
                curr_folder = ""
                endorsed = 0
                
            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Subject"): #We want to check if this is actually the question eventaully
                curr_topic = col
                
            elif(field == "Folders"):
                curr_folder = col
                
            elif(field == "Endorsed by Instructor"):
                if(col == "TRUE"):
                    endorsed = 1
                else:
                    endorsed = 0
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    if(already_filled == 0):
                        curr_question = curr_text
                    already_filled = 0
                elif(col == "updated_question"):
                    # update question entry to post and follow up numbers
                    if(already_filled == 0):
                        curr_question = curr_text
                        already_filled = 1
                        
                elif(col == "updated_i_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                    endorsed = 1
                elif(col == "started_off_i_answer"):
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    endorsed = 1
                    
                elif(col == "updated_s_answer"):
                    # update last entry to the post and follow up number
                    # what if it was appended? how do we update just the end?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                        already_filled = 1
                elif(col == "started_off_s_answer"):
                    # append to answer - if it was previously endorsed and were appending should we keep it endorsed?
                    if(already_filled == 0):
                        curr_answer = curr_answer + "\n" + curr_text
                    already_filled = 0
                    
                elif(col == "followup"):
                    # new entry for the new followup question 
                    curr_folders = curr_folder.split()
                    for folder in curr_folders:
                        if folder[-1] == ';':
                            folder = folder[:-1]
                        if folder in raw_questions_to_hw:
                            raw_questions = raw_questions_to_hw[folder]
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                        else:
                            raw_questions = []
                            raw_questions.append(curr_question)
                            raw_questions_to_hw.update({folder : raw_questions})
                    curr_question = ""
                    curr_answer = ""
                    curr_folder = ""
                    endorsed = 0
                    already_filled = 0
                        
                    curr_follow_up = curr_follow_up + 1
                    curr_question = curr_text
                elif(col == "reply_to_followup"):
                    curr_answer = curr_answer + "\n" + curr_text
                    
num = 0
# for i in raw_questions_to_hw["hw5"]:
#     print(num)
#     print(i)
#     print('\n')
#     num = num+1

# myDB.close()

In [201]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from xml.dom import minidom

folder = "hw1"
print(len(raw_questions_to_hw[folder]))

# parse an xml file by name
mydoc = minidom.parse('Posts.xml')

SEitems = mydoc.getElementsByTagName('row')

SEitems_questions = []
for item in SEitems:
    SEitems_questions.append(item.attributes['Body'].value)
    
SEitems_questions = SEitems_questions[1:6000]

vectorizer = TfidfVectorizer(stop_words='english')
X_orig = vectorizer.fit(SEitems_questions)
X = vectorizer.transform(raw_questions_to_hw[folder])

# Number of clusters per homework
true_k = 50
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)

# print("\n")
# print("Prediction")



276


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [202]:
# Combine clusters that have references to each other

Y = vectorizer.transform(["Should we fail for empty queues for delete and contains too?"])
prediction = model.predict(Y)
# print(prediction)

# for x in range(276):
#     if model.labels_[x] == 28:
#         print("Question Number: ") 
#         print(x)
#         print(raw_questions_to_hw[folder][x])

In [203]:
# Create a list of lists for each cluser

final_clusters = {}
for i in range(len(raw_questions_to_hw[folder])):
    label = model.labels_[i]
    final_clusters.setdefault(label, []).append(raw_questions_to_hw[folder][i])
    
print(final_clusters)

{8: ['In the first hw it says to\xa0refer to the FAQ page for an explanation about test cases.\xa0Where can I find the page?\xa0', 'when i submit my hw, it keeps on the loading page saying: "Loading. Please wait..." and it never actually submits my hw. I tried reloading the page and redoing it but it is still not working.', 'I\'m a little unclear on the instructions for problem 8 even after reading the faq page.\xa0 For example, if:\n\nin_both ["Garnet"; "Amethyst"; "Pearl"] ["Pearl"; "Steven"])\n\nThen it should return\n\n ["Pearl"]\nWhat if the string "Pear" appears twice in both lists? Or twice in the second list? Should it return ["Pearl Pear"] in both cases? If so, how should it be formatted?'], 27: ['Every method in problems 1 to 4 has the recursion modifier in front of it.\xa0 Do we need to use recursion\xa0for every problem or can we submit answers without recursion?', 'After doing some problems, I realized that there are some problems that do not need to be solved using recurs

In [204]:
# Shorten the sentences just for one cluster for now
import random
from nltk.tokenize import word_tokenize
import string

number, items = random.choice(list(final_clusters.items()))

stop_words = ["ourselves", "hers", "between", "yourself", "but", "again", 
                  "there", "about", "once", "during", "out", "very", "having", 
                  "with", "they", "own", "an", "be", "some", "for", "do", "its", 
                  "yours", "such", "into", "of", "most", "itself", "other", "off",
                  "is", "s", "am", "or", "who", "as", "from", "him", "each", "the",
                  "themselves", "until", "below", "are", "we", "these", "your", "his", 
                  "through", "don", "nor", "me", "were", "her", "more", "himself", "this",
                  "down", "should", "our", "their", "while", "above", "both", "up", "to",
                  "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them",
                  "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", 
                  "then", "that", "because", "what", "over", "why", "so", "can", "did", 
                  "not", "now", "under", "he", "you", "herself", "has", "just",
                  "where", "too", "only", "myself", "which", "those", "i", "after",
                  "few", "whom", "t", "being", "if", "theirs", "my", "against", "a",
                  "by", "doing", "it", "how", "further", "was", "here", "than", "ok",
                  "thank", "ah"]

length = len(items)
shortened_questions = []
for i in range(0, length):
    question = items[i]
    tokens = word_tokenize(question)
    tokens = [w.lower() for w in tokens]
    words = [w for w in tokens if (not w in stop_words and w != "" and w != " ")]

    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]

    final_question = " ".join(words)

    shortened_questions.append(final_question)

#     shortened_questions = [s for s in shortened_questions if s != '']

#     # Printing out average
# totalNumWords = 0
# maxLength = 0
# for s in shortened_questions:
#     numWords = len(word_tokenize(s))
#     if (maxLength < numWords): 
#         maxLength = numWords
#     # print(numWords)
#     if (numWords > 50):
#         print(s)
#         print("\n")
#     totalNumWords += numWords

print(shortened_questions)



['m going kudos problem  remove  permutation  function introml type signature intromli ', 'submitting introml zip ']


In [205]:
from bert_serving.client import BertClient 

# Start BERT client for mrpc bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/mrpc_output -num_worker=4 -max_seq_len=50
# Start BERT client for qqa checkpoint bert-serving-start -model_dir /Users/maralevy/Downloads/uncased_L-12_H-768_A-12 -tuned_model_dir /Users/maralevy/Documents/Piazza-SeniorDesign/bert/qqp_output -num_worker=4 -max_seq_len=50 -ckpt_name bert_model.ckpt-1000
bc = BertClient(check_version=False)  
doc_vecs = bc.encode(shortened_questions)

In [206]:
# This cell allows you to type a question and find semantically similar questions using BURT, eventually
# we should run this for every question in raw_questions

import np
print(doc_vecs)
topk = len(shortened_questions)
final_scores = {}
for question in shortened_questions:
    query_vec = bc.encode([question])
    # compute simple dot product as score
    score = np.sum(query_vec * doc_vecs, axis=1)/np.linalg.norm(doc_vecs, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
#     print("Question: ")
#     print(question)
    for idx in topk_idx:
        if idx != topk_idx[0]:
            final_scores.setdefault(items[topk_idx[0]], []).append((score[idx], items[idx]))

print(final_scores)

[[-1.4484812   0.29755905  0.19067964 ...  0.04615161 -0.95012856
   0.9246801 ]
 [-0.8475524   0.0053727  -0.17096393 ...  0.32670292 -0.48785943
   0.20569578]]
{'If I\'m not going to do the Kudos problem, should I remove the "permutation" function from intro.ml and its type\xa0signature from intro.mli?': [(17.134724, 'Are we just submitting intro.ml as a zip?')], 'Are we just submitting intro.ml as a zip?': [(13.466512, 'If I\'m not going to do the Kudos problem, should I remove the "permutation" function from intro.ml and its type\xa0signature from intro.mli?')]}


In [207]:
cluster_number = 0
final_clusters = {}
used_strings = {}
for key in final_scores:
    quota = float(final_scores[final_scores[key][0][1]][0][0] - final_scores[final_scores[key][0][1]][1][0])
    if key == final_scores[final_scores[key][0][1]][0][1] or \
    (key == final_scores[final_scores[key][0][1]][1][1] and \
     (abs(final_scores[final_scores[key][0][1]][1][0] - final_scores[final_scores[key][0][1]][0][0]) < quota * 2)) or \
    (key == final_scores[final_scores[key][0][1]][2][1] and \
     (abs(final_scores[final_scores[key][0][1]][2][0] - final_scores[final_scores[key][0][1]][0][0]) < quota * 3)):        
        # Combine clusters if needed
        if key in used_strings and final_scores[key][0][1] in used_strings:
            cluster_number_one = used_strings[key]
            cluster_number_two = used_strings[final_scores[key][0][1]]
            if cluster_number_two != cluster_number_one:
                final_clusters[cluster_number_one] = final_clusters[cluster_number_one] + final_clusters[cluster_number_two]
                for string in final_clusters[cluster_number_two]:
                    used_strings[string] = cluster_number_one
                final_clusters.pop(cluster_number_two, None)
        elif key in used_strings:
            final_clusters[used_strings[key]].append(final_scores[key][0][1])
        elif final_scores[key][0][1] in used_strings:
            final_clusters[used_strings[final_scores[key][0][1]]].append(key)
        else:
            used_strings[key] = cluster_number
            used_strings[final_scores[key][0][1]] = cluster_number
            final_clusters[cluster_number] = [key, final_scores[key][0][1]]
            cluster_number = cluster_number + 1
            
for key in final_clusters:
    print("Cluster %d:"% key)
    for q in final_clusters[key]:
        print("Q: \n %s" % q)
        


IndexError: list index out of range